# 블로그 컨텐츠 수집(전체)

데이터가 대량으로 수집되어야 하는 경우 일부에 대한 처리 코드를 완성하고 이를 모듈화(함수)하여 재사용하면 대량의 데이터 수집이 가능하다

## #01. 준비과정
### [1] 패키지 참조


In [9]:
import requests
from bs4 import BeautifulSoup
import datetime as dt

## #02. 데이터 요청하기
### [1] 세션요청

In [10]:
session = requests.Session()

session.headers.update({
    "Referer": "",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
})

### [2] 웹 페이지 소스코드 가져오기

1. 앞서 구현한 코드를 모두 하나의 블록으로 모은다.
2. 모아놓은 코드를 함수로 묶는다. 
   - 접근URL을 파라미터로 처리한다.
3. 함수 안에서 생성된 결과값은 리턴

In [11]:
def getContents(session, url):
    try:
        r = session.get(url)
        
        if r.status_code != 200:
            msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
            raise Exception(msg)
    except Exception as e:
        print("접속에 실패했습니다.")
        print(e)


    r.encoding = "utf-8"
    soup = BeautifulSoup(r.text)

    post = soup.select('.post')
    mydata =[]
    for i, v in enumerate(post):
        # print(v)

        # 하나의 글 안에서 제목 영역을 찾는다.
        entryTitle = v.select(".entry-title a")
        # print(entryTitle)

        # 추출된 요소가 각 게시글 안에서 하나만 존재하므로 `0`번째 원소에 직접 접근한다.
        title = entryTitle[0].text.strip()
        # print(title)

        # 클릭시 이동할 페이지의 주소
        if 'href' in entryTitle[0].attrs:
            href = entryTitle[0].attrs['href']

        # 같은 페이지 내에서 주소 이동할 경우 도메인 생략가능
            
        # 수집된 주소에 도메인이 없다면 덧붙여준다
        if url not in href:
            href = url+href

        else:
            href = None
        # print(href)

        # 작성일
        published = v.select(".published")
        # print(published)
        datetime = published[0].attrs['datetime']
        # print(datetime)
        
        # 요약글
        entryContent = v.select(".entry-content p")
        # print(entryContent)

        # 마지막의 `more` 버튼은 제거한다.
        # entryContent = entryContent[:-1] # python적 접근
        # print(entryContent)

        entryContent = v.select(".entry-content p:not(.read-more)") # CSS적 접근
        # print(entryContent)

        text = ''
        for j, e in enumerate(entryContent): # 2개 이상인 경우도 있으므로 반복문
            text += " " +e.text.strip() if j>0 else e.text.strip()
        # print(text)

        # 태그
        tagLinks = v.select(".tag-links a")
        # print(tagLinks)

        
        for j,e in enumerate(tagLinks):
            tagLinks[j] = e.text.strip()
        
        tags = ','.join(tagLinks)
        
        # 수집된 정보를 하나의 딕셔너리로 묶는다 => 처리방법은 다르게해도 상관 없음
        mydict = {
            "title":title,
            "href":href,
            "datetime":datetime,
            "text":text,
            "tags":tags
        }
        mydata.append(mydict)
    return mydata


### [3] 구현된 기능 테스트하기

In [13]:
result = getContents(session, 'https://blog.hossam.kr')
result

### [4] 반복문에서 주소 패턴을 구성하여 수행
#### 접근 URL의 패턴 정의

In [14]:
url = "https://blog.hossam.kr"
urlFmt = "%s/blog/page{pagenumber}" %url


#### 최대 페이지 수 정의

In [15]:
maxPage = 22


#### 데이터 가져오기

In [19]:
blogPosts = []
for i in range(1,maxPage+1):
    targetUrl = urlFmt.format(pagenumber = i) if i>1 else url
    blogPosts += getContents(session,targetUrl)

print("수집된 전체 게시글 수 :",len(blogPosts))

수집된 전체 게시글 수 : 108


### [5] 수집 결과 저장하기

In [20]:
fname = dt.datetime.now().strftime("블로그_글_수집_%y%m%d_%H%M%S.csv")

with open(fname, 'w', encoding='utf-8') as f:
    for i,v in enumerate(blogPosts):
        if i ==0 : 
            title = f'{",".join(v.keys())}\n'
            # print(title)
            f.write(title)
        
        content = list(v.values())

        # print(content)
        # csv에 저장하기 위해서
        # 1) 각 컨텐츠 안에 포함된 쌍따옴표는 역슬래시로 묶어준다
        # 2) 각 컨텐츠를 쌍따옴표로 묶어준다.
        for j,w in enumerate(content):
            content[j] = f'"{w.replace('"',r'\"')}"'
        # print(content)
        f.write(f'{",".join(content)}\n')